In [ ]:
import os
from scipy.ndimage import zoom
import nibabel as nib
import numpy as np

### Delete slices that have less than 4 class


In [ ]:
def get_patient_ids(dataset_path):
    """Extracts patient IDs from the folder names dynamically."""
    patient_ids = []
    for folder in os.listdir(dataset_path):
        if folder.startswith("patient"):
            try:
                patient_id = int(folder.replace("patient", ""))
                patient_ids.append(patient_id)
            except ValueError:
                continue 
    return sorted(patient_ids)

def filter_slices_with_min_classes(dataset_type, output_root, min_classes=4):
    dataset_path = os.path.join("just_to_test", dataset_type)
    patient_ids = get_patient_ids(dataset_path)  
    
    for patient_id in patient_ids:
        patient_folder = os.path.join(dataset_path, f"patient{patient_id:03d}")
        output_folder = os.path.join(output_root, dataset_type, f"patient{patient_id:03d}")
        os.makedirs(output_folder, exist_ok=True)
        
        files = {
            "ED": f"patient{patient_id:03d}_ED_processed.nii.gz",
            "ED_gt": f"patient{patient_id:03d}_ED_gt_processed.nii.gz",
            "ES": f"patient{patient_id:03d}_ES_processed.nii.gz",
            "ES_gt": f"patient{patient_id:03d}_ES_gt_processed.nii.gz",
        }

        for phase in ["ED", "ES"]:
            image_path = os.path.join(patient_folder, files[phase])
            mask_path = os.path.join(patient_folder, files[phase + "_gt"])
            
            if not os.path.exists(image_path) or not os.path.exists(mask_path):
                continue

            print(f"Processing Patient {patient_id:03d} - {phase}")
            image = nib.load(image_path).get_fdata()
            mask = nib.load(mask_path).get_fdata()

            num_slices = image.shape[2]
            valid_slices = []
            
            for z in range(num_slices):
                unique_classes = np.unique(mask[:, :, z])
                if len(unique_classes) >= min_classes:
                    valid_slices.append(z)
            
            if valid_slices:
                
                filtered_image = image[:, :, valid_slices]
                filtered_mask = mask[:, :, valid_slices]
                
                nib.save(nib.Nifti1Image(filtered_image, np.eye(4)), os.path.join(output_folder, files[phase]))
                nib.save(nib.Nifti1Image(filtered_mask, np.eye(4)), os.path.join(output_folder, files[phase + "_gt"]))
                print(f"Saved filtered NIfTI files for Patient {patient_id:03d} - {phase} in {output_folder}")
            else:
                print(f"No valid slices with >= {min_classes} classes found for Patient {patient_id:03d} - {phase}")

filter_slices_with_min_classes("training", "filtered_slice", min_classes=4)


Processing Patient 001 - ED
Saved filtered NIfTI files for Patient 001 - ED in filtered_slice\training\patient001
Processing Patient 001 - ES
Saved filtered NIfTI files for Patient 001 - ES in filtered_slice\training\patient001
Processing Patient 002 - ED
Saved filtered NIfTI files for Patient 002 - ED in filtered_slice\training\patient002
Processing Patient 002 - ES
Saved filtered NIfTI files for Patient 002 - ES in filtered_slice\training\patient002
Processing Patient 003 - ED
Saved filtered NIfTI files for Patient 003 - ED in filtered_slice\training\patient003
Processing Patient 003 - ES
Saved filtered NIfTI files for Patient 003 - ES in filtered_slice\training\patient003
Processing Patient 004 - ED
Saved filtered NIfTI files for Patient 004 - ED in filtered_slice\training\patient004
Processing Patient 004 - ES
Saved filtered NIfTI files for Patient 004 - ES in filtered_slice\training\patient004
Processing Patient 005 - ED
Saved filtered NIfTI files for Patient 005 - ED in filtered_

### ROI crop

In [ ]:
import os
import numpy as np
import nibabel as nib
from scipy.ndimage import zoom

def get_patient_ids(dataset_path):
    """Extracts patient IDs from the folder names dynamically."""
    patient_ids = []
    for folder in os.listdir(dataset_path):
        if folder.startswith("patient"):
            try:
                patient_id = int(folder.replace("patient", ""))
                patient_ids.append(patient_id)
            except ValueError:
                continue  
    return sorted(patient_ids)

def resize_image(image, target_size=352):
    """Resizes the image to the target size while maintaining depth."""
    scale_x = target_size / image.shape[0]
    scale_y = target_size / image.shape[1]
    return zoom(image, (scale_x, scale_y, 1), order=1) 

def crop_center(image, target_size=224):
    """Crops the center of the image to the specified target size."""
    x_center, y_center = image.shape[:2]
    x_start = (x_center - target_size) // 2
    y_start = (y_center - target_size) // 2
    return image[x_start:x_start+target_size, y_start:y_start+target_size, :]

def process_and_crop_slices(dataset_type, output_root):
    dataset_path = os.path.join("filtered_slice", dataset_type)
    patient_ids = get_patient_ids(dataset_path)  
    
    for patient_id in patient_ids:
        patient_folder = os.path.join(dataset_path, f"patient{patient_id:03d}")
        output_folder = os.path.join(output_root, dataset_type, f"patient{patient_id:03d}")
        os.makedirs(output_folder, exist_ok=True)
        
        files = {
            "ED": f"patient{patient_id:03d}_ED_processed.nii.gz",
            "ED_gt": f"patient{patient_id:03d}_ED_gt_processed.nii.gz",
            "ES": f"patient{patient_id:03d}_ES_processed.nii.gz",
            "ES_gt": f"patient{patient_id:03d}_ES_gt_processed.nii.gz",
        }

        for phase in ["ED", "ES"]:
            image_path = os.path.join(patient_folder, files[phase])
            mask_path = os.path.join(patient_folder, files[phase + "_gt"])
            
            if not os.path.exists(image_path) or not os.path.exists(mask_path):
                continue

            print(f"Processing Patient {patient_id:03d} - {phase}")
            image = nib.load(image_path).get_fdata()
            mask = nib.load(mask_path).get_fdata()

            resized_image = resize_image(image, 352)
            resized_mask = resize_image(mask, 352)

            cropped_image = crop_center(resized_image, 224)
            cropped_mask = crop_center(resized_mask, 224)
            
            nib.save(nib.Nifti1Image(cropped_image, np.eye(4)), os.path.join(output_folder, files[phase]))
            nib.save(nib.Nifti1Image(cropped_mask, np.eye(4)), os.path.join(output_folder, files[phase + "_gt"]))
            print(f"Saved cropped NIfTI files for Patient {patient_id:03d} - {phase} in {output_folder}")

process_and_crop_slices("traing", "cropped_slices_processed")


Processing Patient 101 - ED
Saved cropped NIfTI files for Patient 101 - ED in cropped_slices_processed\testing\patient101
Processing Patient 101 - ES
Saved cropped NIfTI files for Patient 101 - ES in cropped_slices_processed\testing\patient101
Processing Patient 102 - ED
Saved cropped NIfTI files for Patient 102 - ED in cropped_slices_processed\testing\patient102
Processing Patient 102 - ES
Saved cropped NIfTI files for Patient 102 - ES in cropped_slices_processed\testing\patient102
Processing Patient 103 - ED
Saved cropped NIfTI files for Patient 103 - ED in cropped_slices_processed\testing\patient103
Processing Patient 103 - ES
Saved cropped NIfTI files for Patient 103 - ES in cropped_slices_processed\testing\patient103
Processing Patient 104 - ED
Saved cropped NIfTI files for Patient 104 - ED in cropped_slices_processed\testing\patient104
Processing Patient 104 - ES
Saved cropped NIfTI files for Patient 104 - ES in cropped_slices_processed\testing\patient104
Processing Patient 105 -